# Utility notebook for arrival, departure, and TK statistics data

In [1]:
import copy
import pandas as pd
import numpy as np

## Migration and municipality data

In [2]:
mun_stats = pd.read_csv("data/statfin_vaerak_pxt_11ra_municipalities.csv", index_col=0)
mun_migri = pd.read_csv("data/statfin_muutl_pxt_11a1_municipalities.csv", index_col=0)

pop_stats = \
    (mun_stats
     .loc[mun_stats["Information"] == "Population 31 Dec", :]
     .melt(
        id_vars=["Area", "Information"],
        var_name="year",
        value_name="Population 31 Dec")
     .drop(columns=["Information"])
     .pivot(columns="Area", index="year")
     # .astype('int')
     .describe())

muns_of_interest = list(pop_stats.loc[:, (pop_stats.loc[["min"]] > 10000).values[0]].columns.droplevel())

# clean the names

mun_migri["Area of arrival"] = mun_migri["Area of arrival"].str.split('-').map(lambda x: x[-1].strip())
mun_migri["Area of departure"] = mun_migri["Area of departure"].str.split('-').map(lambda x: x[-1].strip())
mun_migri.head()

# make a table like
# Arrival | Departure | Sex | Year

male_migri = mun_migri.loc[:, mun_migri.columns.map(lambda x: "Males" in x)]
male_migri = male_migri.rename(columns={k:(k.split()[1]) for k in male_migri.columns})
male_migri["Arrival"] = mun_migri["Area of arrival"]
male_migri["Departure"] = mun_migri["Area of departure"]

female_migri = mun_migri.loc[:, mun_migri.columns.map(lambda x: "Females" in x)]
female_migri = female_migri.rename(columns={k:(k.split()[1]) for k in female_migri.columns})
female_migri["Arrival"] = mun_migri["Area of arrival"]
female_migri["Departure"] = mun_migri["Area of departure"]

#

female_migri = female_migri.melt(id_vars=["Arrival", "Departure"], 
                                 value_name="Migration", 
                                 var_name="year")
female_migri["Sex"] = "female"

male_migri = male_migri.melt(id_vars=["Arrival", "Departure"], 
                             value_name="Migration", 
                             var_name="year")
male_migri["Sex"] = "male"

#

_s = set(muns_of_interest)
male_migri = male_migri.loc[male_migri["Arrival"].isin(_s), :]
male_migri = male_migri.loc[male_migri["Departure"].isin(_s), :]
male_migri = male_migri.loc[(male_migri["Arrival"] != male_migri["Departure"]), :]
male_migri = male_migri.loc[(male_migri["Migration"] != 0), :]

female_migri = female_migri.loc[female_migri["Arrival"].isin(_s), :]
female_migri = female_migri.loc[female_migri["Departure"].isin(_s), :]
female_migri = female_migri.loc[(female_migri["Arrival"] != female_migri["Departure"]), :]
female_migri = female_migri.loc[(female_migri["Migration"] != 0), :]

# 

total_migri = pd.concat([male_migri, female_migri])

In [3]:
mun_stats_l = mun_stats.melt(id_vars=["Area", "Information"],
                           var_name="year",
                           value_name="value")
mun_stats_l.head()

,Area,Information,year,value
0,Akaa,Population 31 Dec,1990,16048.0
1,Akaa,Increase of population,1990,111.0
2,Akaa,"Increase of population, %",1990,0.7
3,Akaa,"Share of persons aged under 15, %",1990,18.5
4,Akaa,"Share of persons aged 15 to 64, %",1990,65.7


In [4]:
# arrival | sex | year | arr stats | dep stats | arr - dep stats

_t = total_migri[["Arrival", "Departure", "year", "Sex"]].merge(mun_stats_l, left_on=["Arrival", "year"], right_on=["Area", "year"], how="left")
_t = _t.rename(columns={"Information": "Info_arr"})
_t.drop(columns=["Area"], inplace=True)


In [5]:
_t = _t.pivot(columns=["Info_arr"], values="value", index=["Arrival", "Departure", "year", "Sex"])

In [6]:
mun_stats_w = mun_stats.pivot(index=["Area"], columns="Information").stack(level=0)
mun_stats_w.index = mun_stats_w.index.set_names(['Area', 'year'])
mun_stats_w = mun_stats_w.reset_index()

/tmp/ipykernel_6782/161357135.py:1: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  mun_stats_w = mun_stats.pivot(index=["Area"], columns="Information").stack(level=0)


In [7]:
_ti = _t.reset_index()
_w = _ti.merge(mun_stats_w, left_on=["Departure", "year"], right_on=["Area", "year"], how="left", suffixes=["_arr", "_dep"])

full_data = _w.merge(total_migri, on=["Arrival", "Departure", "year", "Sex"])
full_data = full_data.sort_values(by=["Arrival", "year", "Departure", "Sex"])

del _t, _w, _ti

In [8]:
_year_sums = full_data.groupby(["year", "Sex"])["Migration"].sum()
_year_sums = _year_sums.unstack().to_dict()

full_data["Migration_normalized"] = [m / _year_sums[s][y] for (y, s, m) in zip(full_data["year"], full_data["Sex"], full_data["Migration"])]

In [9]:
base_cols = set(col.rsplit('_', 1)[0] for col in full_data.columns if col.endswith('_arr'))

# Calculate differences and add as new columns
for base in base_cols:
    arr_col = f"{base}_arr"
    dep_col = f"{base}_dep"
    diff_col = f"{base}_diff"
    full_data[diff_col] = full_data[arr_col] - full_data[dep_col]

In [10]:
full_data

,Arrival,Departure,year,Sex,"Average age, both sexes_arr","Average age, men_arr","Average age, women_arr",Citizens of foreign countries_arr,Demographic dependency ratio_arr,Economic dependency ratio_arr,...,Persons born elsewhere than in their area of residence_diff,"Share of persons with no religious affiliation, %_diff","Share of persons in rural heartland areas, %_diff",Citizens of foreign countries_diff,"Share of persons in urban areas, %_diff","Share of persons aged under 15, %_diff","Share of persons belonging to the Evangelical Lutheran Church, %_diff","Share of persons born abroad, %_diff","Share of persons in sparsely populated rural areas, %_diff","Share of persons belonging to other religious groups, %_diff"
12,Akaa,Espoo,1990,female,38.6,36.3,40.8,70.0,52.2,130.7,...,-123606.0,-4.3,1.5,-1986.0,-96.6,-3.1,5.0,-1.0,0.0,-0.8
13,Akaa,Espoo,1990,male,38.6,36.3,40.8,70.0,52.2,130.7,...,-123606.0,-4.3,1.5,-1986.0,-96.6,-3.1,5.0,-1.0,0.0,-0.8
78,Akaa,Eura,1990,female,38.6,36.3,40.8,70.0,52.2,130.7,...,3405.0,5.1,-44.6,46.0,-2.1,0.4,-5.8,0.5,-1.2,0.7
79,Akaa,Eura,1990,male,38.6,36.3,40.8,70.0,52.2,130.7,...,3405.0,5.1,-44.6,46.0,-2.1,0.4,-5.8,0.5,-1.2,0.7
95,Akaa,Forssa,1990,female,38.6,36.3,40.8,70.0,52.2,130.7,...,-3061.0,1.3,0.9,13.0,-89.5,0.8,-1.5,0.2,0.0,0.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307890,Äänekoski,Turku,2023,male,47.6,46.2,48.9,390.0,78.0,183.7,...,-118759.0,-12.4,43.5,-18672.0,-98.2,2.0,12.3,-10.3,10.1,0.1
307933,Äänekoski,Ulvila,2023,female,47.6,46.2,48.9,390.0,78.0,183.7,...,-273.0,4.4,43.5,152.0,-88.4,-2.0,-6.2,0.3,9.8,1.9
308013,Äänekoski,Vaasa,2023,female,47.6,46.2,48.9,390.0,78.0,183.7,...,-30521.0,-4.6,43.5,-5496.0,-93.3,0.1,3.6,-8.8,10.1,1.0
308113,Äänekoski,Vantaa,2023,female,47.6,46.2,48.9,390.0,78.0,183.7,...,-177008.0,-20.6,43.5,-41976.0,-97.9,-2.3,21.6,-19.4,10.1,-1.1


In [11]:
# full_data.to_csv("data/full_arr_dep_stats.csv")